# Import Modules and Genome dataset from API

In [20]:
import DAL
import numpy as np 
from numpy import *

genomes = DAL.create("genomes")
genome_list = genomes.subsets()


# Set up multiple working clusters by Ipython.parallel

In [21]:
from IPython.parallel import Client

rc = Client()

import time
while len(rc) < 4: 
    time.sleep(1)


dview = rc[:]
dview.retries = 10

for i in rc.ids:
    rc[i]["id"] = i

# Generate a strand file for each genome

In [22]:

@dview.remote(block=True)
def generate_strandfile():
    import DAL
    genomes = DAL.create("genomes")
    strandfile_ref=[]
    for genome_name in genome_list:
        fp = open("/tmp/strand_file.fa"+str(genome_list.index(genome_name))+str(id), "w")
        strandfile_ref.append(DAL.helpers.FileReference(fp))
        for chunk in genomes.k_mers(genome_name,20):
            fp.write(chunk+" "+genome_name+"\n")
        fp.close
    return strandfile_ref       
    
dview.scatter("genome_list", genome_list)

temp = generate_strandfile()

strandfile_ref = []
for sublist in temp:
    for item in sublist:
        strandfile_ref.append(item)



#As the size of the strandfile is too large for the single working cluster's memory, I split the strand file to 1,000,000-row chunks. Then I could process every chunk once, save the result into disk and free the memory.# 

In [23]:

splitLen = 1000000   
dview["splitLen"] = splitLen
@dview.remote(block=True) 
def sortedfile_ref():
    ssortedfile_ref = []
    import DAL
    import heapq
    from heapq import merge
    for file_ref in strandfile_ref:
        sorted_file = []
        temp_file = file_ref.open("r")
        input_data = temp_file.readlines()
        temp_file.close
        file_ref.remove_local_copy()
        ttempfile_ref = []
        
        lenrange = range(0, len(input_data), splitLen)
        for start_index in lenrange:
            if start_index+splitLen <= len(input_data):
                end_index = start_index + splitLen
                output_data = input_data[start_index:end_index]
            else:
                output_data = input_data[start_index:-1]
                
            output_data.sort()
            ttemp_file = open(file_ref.filename+".tmp" +str(lenrange.index(start_index)), "w")
            ttempfile_ref.append(DAL.helpers.FileReference(ttemp_file))
            
            for lines in output_data:
                ttemp_file.write(lines)
            ttemp_file.close
            
            
        sortedtemp_file = []    
     
    
        for ffile_ref in ttempfile_ref:
            sortedtemp_file.append(ffile_ref.open("r"))
     
        with open(file_ref.filename+".sorted", "w") as r:
            ssortedfile_ref.append(DAL.helpers.FileReference(r))
            for chunk in merge(*sortedtemp_file):
                r.write(chunk)  
        
        for ffile_ref in ttempfile_ref:
            ffile_ref.remove_local_copy()
        
        
        for tttemp_file in sortedtemp_file:
            tttemp_file.close()
   
            
    ssortedtemp_file = [] 
    for fffile_ref in ssortedfile_ref:
        ssortedtemp_file.append(fffile_ref.open("r"))
    
    with open("/tmp/submerge_file.fa" + str(id), "w" ) as m:
        submergedfile_ref = DAL.helpers.FileReference(m)
        for cchunk in merge(*ssortedtemp_file):
            m.write(cchunk)
            
    for fffile_ref in ssortedfile_ref:
        fffile_ref.remove_local_copy()
            
    for ttttemp_file in ssortedtemp_file:
            ttttemp_file.close()
    
      
    return submergedfile_ref 
     
dview.scatter("strandfile_ref", strandfile_ref)
sortedfile_ref = sortedfile_ref()

#I am going to combine all the strand files to one file by merge in the heapq module due to the limitation of memory

In [24]:

import heapq
from heapq import merge
temp_file = []
for file_ref in sortedfile_ref:
    temp_file.append(file_ref.open("r"))

with open("/tmp/merged_file.fa", "w" ) as f:
        mergedfile_ref = DAL.helpers.FileReference(f)
        for chunk in merge(*temp_file):
            f.write(chunk)    

for file_ref in sortedfile_ref:
    file_ref.remove_local_copy()
            
for ttemp_file in temp_file:
    ttemp_file.close()

#print first few lines to test:

In [25]:
temp_file = mergedfile_ref.open("r")
input_data = temp_file.readlines()
temp_file.close()
mergedfile_ref.remove_local_copy()
print array(input_data[:10])

['AAAAAAAAAAAAATTACCCC L._Ipsum-strain00.fa\n'
 'AAAAAAAAAAAAATTACCCC L._Ipsum-strain03.fa\n'
 'AAAAAAAAAAAATTACCCCT L._Ipsum-strain00.fa\n'
 'AAAAAAAAAAAATTACCCCT L._Ipsum-strain03.fa\n'
 'AAAAAAAAAAAGATTACCCC L._Ipsum-strain02.fa\n'
 'AAAAAAAAAAAGGAAACTTA L._Ipsum-strain01.fa\n'
 'AAAAAAAAAAAGGGACTAGC L._Ipsum-strain02.fa\n'
 'AAAAAAAAAAAGGGACTAGG L._Ipsum-strain00.fa\n'
 'AAAAAAAAAAAGGGACTAGG L._Ipsum-strain03.fa\n'
 'AAAAAAAAAAAGTACATAGC L._Ipsum-strain03.fa\n']


#function to split the all_chunk to smaller files. And without split the strand has same genome into two files with the size 400,000.

In [26]:
splitLen = 400000 

start_index = 0
end_index = start_index + splitLen
count = 0
temp_ref = []


while end_index <= len(input_data):
    
    
    if input_data[end_index-1][:20] ==  input_data[end_index][:20]:
        end_index += 1
    
    output_data = input_data[start_index:end_index]
    with open("/tmp/temp_all%s.fa" %count, "w") as r:
        temp_ref.append(DAL.helpers.FileReference(r))
        for lines in output_data:
            r.write(lines)
    count += 1
    start_index = end_index
    end_index = start_index + splitLen

    
output_data = input_data[start_index:-1]
with open("/tmp/temp_all%s.fa" %count, "w") as r:
    temp_ref.append(DAL.helpers.FileReference(r))
    for lines in output_data:
        r.write(lines)    

# function to find-out the similarity matrix 

In [27]:


def sim_mat(x):    ## input argument would be a reference
    import DAL
    import numpy as np
    simmat = np.zeros((n,n))
    simmat = simmat.tolist()
    temp = x.open("r")
    input_data = temp.readlines()
    temp.close()
    start_index = 0
    end_index = start_index + 1
    while end_index < len(input_data):
        [temp_genome,temp_strand] = input_data[start_index].split(" ")
        [test_genome,test_strand] = input_data[end_index].split(" ")
        row_index = int(''.join(ele for ele in temp_strand if ele.isdigit()))
    
        while temp_genome == test_genome:
            col_index = int(''.join(ele for ele in test_strand if ele.isdigit()))
            if col_index != row_index:
                simmat[row_index][col_index] += 1
                simmat[col_index][row_index] += 1
            end_index += 1
            if end_index >= len(input_data):
                break
            [test_genome,test_strand] = input_data[end_index].split(" ")
        start_index = end_index
        end_index = start_index + 1
    return np.array(simmat)    

# Print out the Similarity_Matrix

In [28]:

n = len(genome_list)

dview["n"] = n
simmat = sum(dview.map_sync(sim_mat,temp_ref), axis=0)

print "Here is my Similarity-Matrix:" + "\n"
print simmat

Here is my Similarity-Matrix:

[[       0.  1709899.  1713976.  2426326.]
 [ 1709899.        0.  1490516.   866287.]
 [ 1713976.  1490516.        0.   457548.]
 [ 2426326.   866287.   457548.        0.]]


#For this part, I would firstly gourp the genomes with the maximum similarity. The maximum similarity represents the smallest DNA distance, which means the two genomes must be closely realated. After merge the pair of genome has the maximum similarity, I need to update other elements in the sorted list. 

#For example, I merge pair (0,3), for other pair like (0,1), (3,1). I would choose the pair has better similarity to update, for example if (0,1) > (3,1), new((0,1)) = old((0,1)). Base on my understanding, the more ancient spices would have better similar to all other spices.


In [29]:
temp = zeros((n,n))
temp = triu(simmat)[:]
ttemp = temp[np.nonzero(temp)]
tttemp = sorted(ttemp, reverse = True)
result = [np.argwhere(temp == elements) for elements in tttemp]
pair_list = [ele[0].tolist() for ele in result]

print "Here is sorted list of pair of genomes :" + "\n"
print pair_list


Here is sorted list of pair of genomes :

[[0, 3], [0, 2], [0, 1], [1, 2], [1, 3], [2, 3]]


In [30]:

temp_pair = zip(tttemp,pair_list )
target = temp_pair[0][1]
events = []
while len(temp_pair) > 3:
    
    
        
    temp_seq = range(n)
    target = temp_pair[0][1]
    [a,b]  = target
    c = min(target)

    ttemp_seq = temp_seq[:]
    ttemp_seq.pop(ttemp_seq.index(a))
    ttemp_seq.pop(ttemp_seq.index(b))



    keeppair_list=[]
    for ele in pair_list:
        [s,t] = ele
        if s != a and s != b and t != a and t != b:
            keeppair_list.append(ele)
    
    
    tempkeeppair_list = []        
    for tele in keeppair_list:
        for ttele in temp_pair:
            [g,h] = ttele
            if h == tele:
                tempkeeppair_list.append(ttele)
                           
    for num in ttemp_seq:
        test_list = [[a,num],[num,a],[num,b],[b,num]]
        ttest_list = []
        for coord in test_list:
            ttest_list.append([o for o,p in temp_pair if p == coord])

        temp_index = argmax(ttest_list)
        target_pair = test_list[argmax(ttest_list)]
        keep_number = temp_pair[pair_list.index(target_pair)][0]
        keep_pair = [c, num]
        keeppair_list.append(keep_pair)
        tempkeep_pair = (keep_number, keep_pair)
        tempkeeppair_list.append(tempkeep_pair)


    temp_pair = sorted(tempkeeppair_list, key=lambda tempkeeppair: tempkeeppair[0],reverse = True)
    pair_list = [j for i,j in temp_pair]
    events.append("strain%s branched into strain%s and strain%s " %(c, a, b))
    
if len(temp_pair) == 3:
    target = temp_pair[0][1]
    [a,b]  = target
    c = min(target)
    tempkeeppair_list = []
    (e,f)=temp_pair[1]
    for nnum in f:
        if nnum != a and nnum != b: 
            tempkeeppair_list.append((e,[c, nnum]))
    temp_pair = sorted(tempkeeppair_list, key=lambda tempkeeppair: tempkeeppair[0],reverse = True)
    pair_list = [j for i,j in temp_pair]
    events.append("strain%s branched into strain%s and strain%s " %(c, a, b))

if len(temp_pair) == 1:
    target = temp_pair[0][1]
    [a,b]  = target
    c = min(target)
    events.append("strain%s branched into strain%s and strain%s " %(c, a, b))
    


# Print out the merge events of evolution in order to reconstruct the Cladogram

In [31]:
for log in events[::-1]:
    print log

strain0 branched into strain0 and strain1 
strain0 branched into strain0 and strain2 
strain0 branched into strain0 and strain3 


From the record of grouping the genomes, as the we are grouping the genome from the lower level to higher level. Therefore, we shuold print out all events in reverse, which would be in chronological order. 